In [ ]:
import os
import sys
import json
from pathlib import Path
from typing import Dict, Any, List, Union, Optional
from src.ts_utils import construct_json
from src.ts_main import run_ts, parse_input_dict

In [ ]:
TS_BASE_DIR = ".."

# Example 1

In [ ]:
reag_smiles = [Path(TS_BASE_DIR) / "data" / "aminobenzoic_100.smi",
               Path(TS_BASE_DIR) / "data" / "primary_amines_100.smi",
               Path(TS_BASE_DIR) / "data" / "carboxylic_acids_100.smi"]
reaction = "N[c:4][c:3]C(O)=O.[#6:1][NH2].[#6:2]C(=O)[OH]>>[C:2]c1n[c:4][c:3]c(=O)n1[C:1]"
evaluator_args = {"query_smiles" : "COc1cccc2c(=O)n([C@@H]3C[C@@H](O)[C@@H](O)C3)c([C@@H](N)CNC(N)=O)nc12"}

quinazoline_dict = construct_json(reag_smiles, reaction, evaluator_args=evaluator_args)

In [ ]:
parse_input_dict(quinazoline_dict)
q_df = run_ts(quinazoline_dict, inline_log=True, save_output=False)

# Example 2

In [ ]:
reag_smiles: List[Path] = [
    Path(TS_BASE_DIR) / "data" / "data_ric" / "ald.smi",
    Path(TS_BASE_DIR) / "data" / "data_ric" / "ammines.smi"
]
reaction: Path = Path(TS_BASE_DIR) / "data" / "data_ric" / "reaction.rxn"
evaluator_args = {"query_smiles" : "OC(=O)c1ccc2nc(CN3CCC(CC3)c3cccc(OCc4ccc(cc4F)C#N)n3)n(C[C@@H]3CCO3)c2c1"}

custom_dict = construct_json(reag_smiles, reaction, evaluator_args=evaluator_args)

In [ ]:
parse_input_dict(custom_dict)
c_df = run_ts(custom_dict, inline_log=True, save_output=False)

# Example 3

In [ ]:
reag_smiles: List[Path] = [
    Path(TS_BASE_DIR) / "data" / "data_ric" / "ald.smi",
    Path(TS_BASE_DIR) / "data" / "data_ric" / "ammines.smi"
]

reaction: Path = Path(TS_BASE_DIR) / "data" / "data_ric" / "reaction.rxn"

probe_sdf = Path(TS_BASE_DIR) / "data" / "data_ric" / "crystal.sdf"
evaluator_args = {"query_molfile" : str(probe_sdf)}

custom_dict = construct_json(reag_smiles, reaction, evaluator_args=evaluator_args, evaluator_class="ROCSEvaluator")

In [ ]:
parse_input_dict(custom_dict)

In [ ]:
rocm_df = run_ts(custom_dict, inline_log=True, save_output=False)